# Tic tac toe

> Gradient Tree Boosting implementation test

Imports

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn import tree, preprocessing
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    roc_curve,
    RocCurveDisplay,
    precision_recall_curve,
    PrecisionRecallDisplay,
    f1_score,
)


Parameters

In [7]:
path_file_ds = './file/ds_tic-tac-toe.data'

# Data parsing parameters
class_positive_str, class_negative_str = 'positive', 'negative'
classes_names = [class_positive_str, class_negative_str]
class_positive, class_negative = 1, 0

play_x_str, play_o_str =  'x', 'o'
play_x, play_o = 1, 0

# Hyper parameters
n_folds = 5
test_size = .2
classes_threshold = .5

max_depth = 4
max_trees = 10
random_state = 10
max_features = None
learning_rate = 0.4


## Import data

In [4]:

# Parse input
features = [f'x{i + 1}' for i in range(9)]
col_y = 'y'

df = pd.read_csv(path_file_ds, header=None)
df.columns = features + [col_y]

def play_to_number(x: str):
    return play_x if x == play_x_str else play_o

def class_to_number(y: str):
    return class_positive if y == class_positive_str else class_negative

# Set working data
y = df[col_y].apply(np.vectorize(class_to_number))
X = df[features].apply(np.vectorize(play_to_number))

# See what we've got
n = len(y)
n_positive = np.sum(y == class_positive)
n_negative = np.sum(y == class_negative)

print(f"{n} samples; Positive cases: {n_positive}; Negative cases: {n_negative};")

958 samples; Positive cases: 626; Negative cases: 332;


## Gradient Tree Boosting

In [11]:
class GradientTreeBoosting():
    
    def __init__(
        self, 
        max_depth: int = 4,
        max_trees: int = 10,
        max_features: int = None,
        learning_rate: float = 0.4,
        random_state: int = 0
    ):

        self.max_depth = max_depth
        self.max_trees = max_trees
        self.max_features = max_features
        self.learning_rate = learning_rate
        
        self._weak_learners = []
        self._predictions = np.array([])
        self._random_state = random_state
        

    def fit(self, y: np.array, X: np.array):

        n = len(y)
        predictions = np.array([y.mean()] * n)

        for _ in range(self.max_trees):

            # Creating a weak learner 
            weak_learner = tree.DecisionTreeRegressor(
                max_depth=self.max_depth,
                max_features=self.max_features,
                random_state=self._random_state,
            )

            # Growing the tree on the residuals
            residuals = y - predictions
            weak_learner.fit(X, residuals)
            self._weak_learners.append(weak_learner)

            # Updating predictions
            predictions_wl = weak_learner.predict(X)
            predictions += self.learning_rate * predictions_wl

    def predict(self, X: np.array) -> np.array:
        n = len(X)
        yHat = np.zeros( (n,) )
        for weak_learner in self._weak_learners:
            yHat += self.learning_rate * weak_learner.predict(X)
        return yHat

## Evaluate

In [12]:
classifier = GradientTreeBoosting(
    max_depth=max_depth,
    max_trees=max_trees,
    max_features=max_features,
    learning_rate=learning_rate,
    random_state=random_state,
)

cv = StratifiedShuffleSplit(n_splits=n_folds, test_size=test_size, random_state=random_state)

recalls = np.array([])
accuracies = np.array([])
precisions = np.array([])
f1_scores_pos = np.array([])
f1_scores_neg = np.array([])

i = 0
for train_index, test_index in cv.split(X, y):
    
    # Train & test
    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = y.values[train_index], y.values[test_index]
    n_test, n_train = len(test_index), len(train_index)

    classifier.fit(X=X_train, y=y_train)
    outputs = classifier.predict(X=X_test)
    y_hat = (outputs > classes_threshold).astype(int)

    # Evaluate
    accuracy = np.sum(y_hat == y_test) / n
    f1_pos, f1_neg = f1_score(y_test, y_hat, average=None)
    precision, recall, _ = precision_recall_curve(y_test, outputs, pos_label=1)

    # Display summary
    print('-' * 30)
    print(f"Fold {i + 1}")
    print(r"Train Samples: %d; Test Samples: %d" % (n_train, n_test))
    print(r"Accuracy: %.4f" % accuracy)
    print(r"Precision (average): %.4f" % precision.mean())
    print(r"Recall (average): %.4f" % recall.mean())
    print(r"F1 Score (positive): %.4f" % f1_pos)
    print(r"F1 Score (negative): %.4f" % f1_neg)
    print('')

    # Display confusion Matrix
    fig, (ax1) = plt.subplots(1, 1, figsize=(16, 6))
    conf_matrix = confusion_matrix(y_test, y_hat)
    display_conf_matrix = ConfusionMatrixDisplay(conf_matrix)

    plt.title(f'Confusion Matrix - Iteration {i + 1}', fontsize=22)
    plt.xticks(range(2), classes_names, fontsize=10)
    plt.yticks(range(2), classes_names, fontsize=10)
    plt.xlabel('Predicted', fontsize=12)
    plt.ylabel('Real', fontsize=12)
    display_conf_matrix.plot(ax=ax1)
    plt.show()
    print('')

    # Display ROC Curve
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    fpr, tpr, _ = roc_curve(y_test, outputs, pos_label=1)
    display_roc = RocCurveDisplay(fpr=fpr, tpr=tpr)
    ax1.set_title(f'ROC Curve - Iteration {i + 1}', fontsize=22)
    display_roc.plot(ax=ax1)

    # Display Precision x Recall Curve
    display_precision_recall = PrecisionRecallDisplay(precision=precision, recall=recall)
    ax2.set_title(f'Precision vs. Recall Curve - Iteration {i + 1}', fontsize=22)
    display_precision_recall.plot(ax=ax2)
    plt.show()

    # Compute this fold
    recalls = np.append(recalls, recall)
    accuracies = np.append(accuracies, accuracy)
    precisions = np.append(precisions, precision)
    f1_scores_pos = np.append(f1_scores_pos, f1_pos)
    f1_scores_neg = np.append(f1_scores_neg, f1_neg)
    i += 1

ValueError: Unknown label type: 'continuous'